# Project: Bank Transaction Monitoring and Fraud Detection Pipeline 

# 2. Data Transformation with PySpark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName("BankingTransactionETL-kushi").enableHiveSupport() \
    .getOrCreate()


25/06/26 05:40:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/26 05:40:07 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/26 05:40:07 WARN Utils: Serv

In [4]:
spark

In [6]:
# Load the HDFS data into a PySpark DataFrame.

df_banking = spark.read.option("header",True).option("inferSchema",True).\
csv("/user/ravurisat11dedu/banking/banking_transaction.csv")

In [17]:
df_banking.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- amount: double (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- status: string (nullable = true)



In [18]:
# Total Number of Records.

df_banking.count()

2900

In [19]:
# Dropping rows having null values.

df_banking = df_banking.dropna()

In [20]:
df_banking.count() # No Null values

2900

In [8]:
df_banking.show(10)

+--------------+-----------+-------------------+------+----------------+-------+-------+
|transaction_id|customer_id|          timestamp|amount|transaction_type|channel| status|
+--------------+-----------+-------------------+------+----------------+-------+-------+
|             1|       1082|2025-05-30 22:50:12| 200.2|      withdrawal|    ATM|success|
|             2|       1095|2025-05-26 18:15:58|192.25|         deposit| Branch|success|
|             3|       1004|2025-06-07 21:33:32|184.33|         deposit| Branch|success|
|             4|       1072|2025-06-06 10:26:46|278.95|      withdrawal|    ATM|success|
|             5|       1036|2025-06-05 09:11:09|828.49|         deposit|    ATM|success|
|             6|       1044|2025-06-01 07:42:00|350.08|         deposit| Online|success|
|             7|       1049|2025-05-30 12:32:31|187.04|      withdrawal| Branch|success|
|             8|       1006|2025-06-21 00:31:29|756.76|         deposit| Online|success|
|             9|     

In [22]:
# Perform the following:
# Normalize text fields (transaction_type, channel) to lowercase.
# Filter out failed transactions.
# Add derived columns: txn_day, txn_hour, high_value_flag (if amount > ₹500).


In [23]:
df_banking.select('transaction_type','channel').show(2)

+----------------+-------+
|transaction_type|channel|
+----------------+-------+
|      withdrawal|    ATM|
|         deposit| Branch|
+----------------+-------+
only showing top 2 rows



In [24]:
df_banking.show(5)

+--------------+-----------+-------------------+------+----------------+-------+-------+
|transaction_id|customer_id|          timestamp|amount|transaction_type|channel| status|
+--------------+-----------+-------------------+------+----------------+-------+-------+
|             1|       1082|2025-05-30 22:50:12| 200.2|      withdrawal|    ATM|success|
|             2|       1095|2025-05-26 18:15:58|192.25|         deposit| Branch|success|
|             3|       1004|2025-06-07 21:33:32|184.33|         deposit| Branch|success|
|             4|       1072|2025-06-06 10:26:46|278.95|      withdrawal|    ATM|success|
|             5|       1036|2025-06-05 09:11:09|828.49|         deposit|    ATM|success|
+--------------+-----------+-------------------+------+----------------+-------+-------+
only showing top 5 rows



In [9]:
df_clean = df_banking \
    .withColumn("transaction_type", lower(col("transaction_type"))) \
    .withColumn("channel", lower(col("channel"))) \
    .filter(col("status") == "success") \
    .withColumn("txn_day", dayofmonth("timestamp")) \
    .withColumn("txn_hour", hour("timestamp")) \
    .withColumn("high_value_flag", when(col("amount") > 500, 1).otherwise(0))

In [10]:
df_clean.count() # After removing Failed transactions.

2624

In [11]:
df_clean.show(10)

+--------------+-----------+-------------------+------+----------------+-------+-------+-------+--------+---------------+
|transaction_id|customer_id|          timestamp|amount|transaction_type|channel| status|txn_day|txn_hour|high_value_flag|
+--------------+-----------+-------------------+------+----------------+-------+-------+-------+--------+---------------+
|             1|       1082|2025-05-30 22:50:12| 200.2|      withdrawal|    atm|success|     30|      22|              0|
|             2|       1095|2025-05-26 18:15:58|192.25|         deposit| branch|success|     26|      18|              0|
|             3|       1004|2025-06-07 21:33:32|184.33|         deposit| branch|success|      7|      21|              0|
|             4|       1072|2025-06-06 10:26:46|278.95|      withdrawal|    atm|success|      6|      10|              0|
|             5|       1036|2025-06-05 09:11:09|828.49|         deposit|    atm|success|      5|       9|              1|
|             6|       1

In [34]:
#df_clean.write.option("header", True).csv("/user/ravurisat11dedu/banking/cleaned/")

# 3. Hive-Based Reporting (Batch Analytics)

In [35]:
df_banking.write.mode("overwrite").format("parquet").saveAsTable("kushi.raw_transactions")


In [36]:
df_clean.write.mode("overwrite").format("parquet").saveAsTable("kushi.clean_transactions")


In [40]:
# View for raw transactions

df_banking.createOrReplaceTempView("rawtransactionsview")

In [41]:
# View for cleaned transactions

df_clean.createOrReplaceTempView("cleantransactionsview")

In [42]:

spark.sql("select * from rawtransactionsview").show(2)

+--------------+-----------+-------------------+------+----------------+-------+-------+
|transaction_id|customer_id|          timestamp|amount|transaction_type|channel| status|
+--------------+-----------+-------------------+------+----------------+-------+-------+
|             1|       1082|2025-05-30 22:50:12| 200.2|      withdrawal|    ATM|success|
|             2|       1095|2025-05-26 18:15:58|192.25|         deposit| Branch|success|
+--------------+-----------+-------------------+------+----------------+-------+-------+
only showing top 2 rows



In [44]:
spark.sql("select * from cleantransactionsview").show(5)

+--------------+-----------+-------------------+------+----------------+-------+-------+-------+--------+---------------+
|transaction_id|customer_id|          timestamp|amount|transaction_type|channel| status|txn_day|txn_hour|high_value_flag|
+--------------+-----------+-------------------+------+----------------+-------+-------+-------+--------+---------------+
|             1|       1082|2025-05-30 22:50:12| 200.2|      withdrawal|    atm|success|     30|      22|              0|
|             2|       1095|2025-05-26 18:15:58|192.25|         deposit| branch|success|     26|      18|              0|
|             3|       1004|2025-06-07 21:33:32|184.33|         deposit| branch|success|      7|      21|              0|
|             4|       1072|2025-06-06 10:26:46|278.95|      withdrawal|    atm|success|      6|      10|              0|
|             5|       1036|2025-06-05 09:11:09|828.49|         deposit|    atm|success|      5|       9|              1|
+--------------+--------

# Customer Behavior Insights

In [45]:
# 1. Top 5 customers by number of transactions

spark.sql("SELECT customer_id, COUNT(*) as txn_count FROM cleantransactionsview GROUP BY customer_id  \
ORDER BY txn_count DESC LIMIT 5").show()


+-----------+---------+
|customer_id|txn_count|
+-----------+---------+
|       1032|       42|
|       1041|       38|
|       1014|       38|
|       1084|       37|
|       1052|       37|
+-----------+---------+



In [51]:
# 2. Customer with the highest total withdrawal amount

spark.sql("SELECT customer_id, SUM(amount) AS total_withdrawal_amount FROM cleantransactionsview WHERE transaction_type = 'withdrawal' \
GROUP BY customer_id ORDER BY total_withdrawal_amount DESC  LIMIT 2").show()

+-----------+-----------------------+
|customer_id|total_withdrawal_amount|
+-----------+-----------------------+
|       1032|     13811.570000000002|
|       1018|     13513.199999999997|
+-----------+-----------------------+



In [52]:
#3. Monthly transaction volume per customer
spark.sql("SELECT customer_id, month(timestamp) AS month, COUNT(*) AS txn_count FROM cleantransactionsview \
GROUP BY customer_id, month(timestamp) order by customer_id").show()


+-----------+-----+---------+
|customer_id|month|txn_count|
+-----------+-----+---------+
|       1001|    5|        4|
|       1001|    6|       17|
|       1002|    6|       18|
|       1002|    5|        5|
|       1003|    6|       20|
|       1003|    5|        3|
|       1004|    6|       29|
|       1004|    5|        3|
|       1005|    6|       18|
|       1005|    5|        5|
|       1006|    6|       21|
|       1006|    5|       12|
|       1007|    6|       14|
|       1007|    5|        4|
|       1008|    5|        6|
|       1008|    6|       17|
|       1009|    6|       17|
|       1009|    5|        6|
|       1010|    5|        7|
|       1010|    6|       25|
+-----------+-----+---------+
only showing top 20 rows



# Channel & Platform Analytics


In [53]:
spark.sql("select distinct channel from cleantransactionsview").show()

+-------+
|channel|
+-------+
| online|
|    atm|
| branch|
+-------+



In [54]:
# 1. Average transaction amount by channel

spark.sql("SELECT channel, AVG(amount) AS avg_amount FROM cleantransactionsview \
GROUP BY channel").show()


+-------+-----------------+
|channel|       avg_amount|
+-------+-----------------+
| online|550.9932276995308|
|    atm|543.2210638297872|
| branch|563.8054493742886|
+-------+-----------------+



In [55]:
# 2. Success rate of transactions by channel(raw transactions)

spark.sql("select channel,COUNT(*) AS total, \
(SUM(CASE WHEN status = 'success' THEN 1 ELSE 0 END)*100)/count(*) AS success_rate \
from rawtransactionsview GROUP BY channel").show()
          
         


+-------+-----+-----------------+
|channel|total|     success_rate|
+-------+-----+-----------------+
|    ATM|  985|90.65989847715736|
| Branch|  972| 90.4320987654321|
| Online|  943|90.34994697773065|
+-------+-----+-----------------+



# Time-Based Analytics

In [57]:
# 1. Hourly transaction trend (peak hours)

spark.sql("SELECT txn_hour, COUNT(*) AS txn_count FROM cleantransactionsview \
GROUP BY txn_hour ORDER BY txn_hour").show()


+--------+---------+
|txn_hour|txn_count|
+--------+---------+
|       0|      119|
|       1|      118|
|       2|      118|
|       3|       95|
|       4|       99|
|       5|      108|
|       6|      104|
|       7|      123|
|       8|      102|
|       9|      108|
|      10|      113|
|      11|      121|
|      12|      110|
|      13|      115|
|      14|      101|
|      15|       98|
|      16|       97|
|      17|      109|
|      18|      102|
|      19|      123|
+--------+---------+
only showing top 20 rows



In [58]:
# 2. Day-wise total transaction volume and amount

spark.sql("SELECT txn_day, COUNT(*) AS txn_volume, SUM(amount) AS total_txn_amount \
FROM cleantransactionsview GROUP BY txn_day ORDER BY txn_day").show()

+-------+----------+------------------+
|txn_day|txn_volume|  total_txn_amount|
+-------+----------+------------------+
|      1|        88|48719.049999999996|
|      2|        75|39444.049999999996|
|      3|        78|43144.490000000005|
|      4|        69|35842.829999999994|
|      5|        87| 47592.81000000001|
|      6|       102|59603.519999999975|
|      7|        87| 45472.08000000001|
|      8|        94|53767.769999999975|
|      9|        97| 49049.62999999999|
|     10|        92|47463.419999999984|
|     11|        78| 44055.86000000001|
|     12|        99|          56518.86|
|     13|        83| 50373.60999999998|
|     14|        94|          54303.43|
|     15|        88|          51513.83|
|     16|       101|55754.419999999984|
|     17|        90| 47554.04000000001|
|     18|        78| 44862.31000000001|
|     19|        75| 42361.97000000001|
|     20|        91|45290.139999999985|
+-------+----------+------------------+
only showing top 20 rows



# Fraud Pattern Indicators


In [59]:
# 1. Number of high-value transactions (> ₹500) per customer per day

spark.sql("SELECT customer_id, txn_day, COUNT(*) AS high_value_txns_per_customer FROM cleantransactionsview \
WHERE high_value_flag = 1 GROUP BY customer_id, txn_day ").show()


+-----------+-------+----------------------------+
|customer_id|txn_day|high_value_txns_per_customer|
+-----------+-------+----------------------------+
|       1037|     22|                           1|
|       1063|      2|                           1|
|       1088|     31|                           1|
|       1062|     27|                           2|
|       1048|     12|                           1|
|       1070|      5|                           1|
|       1065|      2|                           1|
|       1027|      8|                           1|
|       1049|     18|                           1|
|       1064|      2|                           1|
|       1033|      1|                           1|
|       1019|     27|                           1|
|       1038|      3|                           1|
|       1039|      4|                           1|
|       1060|     27|                           1|
|       1072|     14|                           1|
|       1099|      3|          

In [ ]:
#2. Customers with more than 3 failed transactions in a day

spark.sql("select customer_id,day(timestamp) as txn_day,count(*) as failed_count from rawtransactionsview \
where status = 'failed' group by customer_id,day(timestamp) having count(*) > 3 ").show()




In [61]:
#spark.sql("select * from cleantransactionsview").show()

In [108]:
spark.sql("SELECT customer_id,channel FROM cleantransactionsview \
WHERE (timestamp >= current_timestamp() - interval 7 days) group by customer_id,channel order by customer_id").show()

+-----------+-------+
|customer_id|channel|
+-----------+-------+
|       1001|    atm|
|       1001| online|
|       1001| branch|
|       1002| online|
|       1002|    atm|
|       1002| branch|
|       1003| branch|
|       1003| online|
|       1004| online|
|       1004| branch|
|       1005|    atm|
|       1006| online|
|       1006| branch|
|       1007| online|
|       1007| branch|
|       1008|    atm|
|       1008| online|
|       1008| branch|
|       1009| branch|
|       1009| online|
+-----------+-------+
only showing top 20 rows



In [62]:
# 3. Customers who used all three channels (ATM, Online, Branch) in the last 7 days

spark.sql(" SELECT customer_id FROM (SELECT customer_id, channel FROM cleantransactionsview \
WHERE timestamp >= current_timestamp() - interval 7 days) GROUP BY customer_id HAVING COUNT(DISTINCT channel) = 3 ").show()          


+-----------+
|customer_id|
+-----------+
|       1088|
|       1025|
|       1016|
|       1064|
|       1034|
|       1056|
|       1093|
|       1046|
|       1077|
|       1065|
|       1061|
|       1055|
|       1008|
|       1047|
|       1100|
|       1062|
|       1021|
|       1026|
|       1074|
|       1029|
+-----------+
only showing top 20 rows

